In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import functools
import xarray as xr
import pandas as pd
import numpy as np

Make an imagestack whose data we can reshape and steal for our object

In [29]:
from starfish.pipeline.features.spots.detector.gaussian import GaussianSpotDetector
from starfish.constants import Indices
from starfish.util.synthesize import synthesize
from starfish.pipeline.features.intensity_table import IntensityTable
from starfish.pipeline.features.codebook import Codebook

data, codebook = synthesize(num_ch=4)
data.image.numpy_array.shape

(4, 4, 1, 100, 100)

In [30]:
gsd = GaussianSpotDetector(blobs_image_name='dots', min_sigma=2, max_sigma=10, num_sigma=10, threshold=0.1)
intensity_table = gsd.find(data)
xr_codebook = Codebook.from_code_array(codebook, 4, 4)

In [21]:
xr_codebook.decode(intensity_table)

KeyError: 'features'

In [32]:
results = xr_codebook.decode(intensity_table)

In [33]:
results

<xarray.IntensityTable (features: 18, c: 4, h: 4)>
array([[[ 2915, 25155,  3346, 15654],
        [30986,  2699,  2915,  2915],
        [ 3346,  3023,  2483,  3023],
        [ 2807,  3130,  2915, 24508]],

       [[24724,  2915, 18354, 19433],
        [ 2699,  2915, 22024,  2483],
        [ 3130, 18246,  2591,  2699],
        [ 2591, 16626, 34872,  2591]],

       ...,

       [[ 2915,  2699, 18138,  2915],
        [ 2915, 25587, 20081,  2807],
        [ 2807,  3238, 18893,  3023],
        [ 2699,  3130,  2807,  2807]],

       [[ 2699,  3023,  3023,  2699],
        [ 2483,  3023, 21377,  2699],
        [ 2915,  2699, 21809, 24616],
        [ 2591,  2699,  2807, 24724]]])
Coordinates:
  * features  (features) MultiIndex
  - x         (features) int64 97 89 87 73 72 68 56 49 49 43 42 37 36 32 25 ...
  - y         (features) int64 60 83 4 65 0 38 20 88 9 2 14 19 25 0 46 38 86 59
  - r         (features) float64 2.828 2.828 2.828 2.828 4.086 2.828 2.828 ...
  - x_min     (features) float64

In [37]:
results.indexes['features'].get_level_values('gene')

Int64Index([77, 87, 27, 77, 74, 94, 106, 77, 32, 110, 21, 46, 103, 39, 85, 108,
            11, 4],
           dtype='int64', name='gene')

In [34]:
xr_codebook.loc[77]

<xarray.Codebook (gene_name: 2, c: 4, h: 4)>
array([[[0, 1, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1]],

       [[0, 1, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1]]], dtype=uint8)
Coordinates:
  * gene_name  (gene_name) int64 77 77
  * c          (c) int64 0 1 2 3
  * h          (h) int64 0 1 2 3

In [144]:
norm_intensities = intensity_table.groupby('features').apply(lambda x: x / x.sum())
norm_codes = xr_codebook.groupby('codes').apply(lambda x: x / x.sum())

In [148]:
np.sqrt(np.sum((norm_intensities[0] - norm_codes[14]) ** 2))

<xarray.DataArray ()>
array(0.162127)
Coordinates:
    features  object (97, 60, 2.8284271247461903, 94.0, 100.0, 57.0, 63.0)
    codes     int64 77

In [156]:
np.sqrt(((norm_intensities[0] - norm_codes) ** 2).groupby('codes').apply(np.sum))

<xarray.DataArray (codes: 19)>
array([0.521081, 0.469842, 0.591049, 0.594157, 0.515162, 0.519904, 0.55237 ,
       0.594498, 0.549771, 0.444764, 0.229283, 0.399665, 0.551253, 0.520691,
       0.593122, 0.592776, 0.592431, 0.493228, 0.495293])
Coordinates:
    features  object (97, 60, 2.8284271247461903, 94.0, 100.0, 57.0, 63.0)
  * codes     (codes) int64 4 7 10 11 21 27 32 39 46 74 77 85 87 94 103 106 ...

In [ ]:
# could also edit values in place

In [15]:
func = functools.partial(min_euclidean_distance, codes=norm_codes)
distances = norm_intensities.groupby('features').apply(func)

In [35]:
min_codes = distances.where(distances == distances.min('codes'))
selected_codes_index = np.ceil(max_codes)

In [41]:
distances.argmin('codes')

<xarray.DataArray (features: 18)>
array([ 0,  3, 16, 11,  7, 14,  8,  4, 18,  6, 10, 15, 13,  9,  1,  5, 12, 10])
Coordinates:
  * features          (features) MultiIndex
  - features_level_0  (features) int64 14 20 25 25 32 36 37 42 43 49 49 56 ...
  - features_level_1  (features) int64 59 86 38 46 0 25 19 14 2 9 88 20 38 0 ...
  - features_level_2  (features) float64 2.828 2.828 2.828 2.828 2.828 2.828 ...
  - features_level_3  (features) float64 11.0 17.0 22.0 22.0 29.0 33.0 34.0 ...
  - features_level_4  (features) float64 17.0 23.0 28.0 28.0 35.0 39.0 40.0 ...
  - features_level_5  (features) float64 56.0 83.0 35.0 43.0 0.0 22.0 16.0 ...
  - features_level_6  (features) float64 62.0 89.0 41.0 49.0 3.0 28.0 22.0 ...
  - features_level_7  (features) object nan nan nan nan nan nan nan nan nan ...
  - features_level_8  (features) object nan nan nan nan nan nan nan nan nan ...

In [40]:
min_codes.argmin('codes')

<xarray.DataArray (features: 18)>
array([ 0,  3, 16, 11,  7, 14,  8,  4, 18,  6, 10, 15, 13,  9,  1,  5, 12, 10])
Coordinates:
  * features          (features) MultiIndex
  - features_level_0  (features) int64 14 20 25 25 32 36 37 42 43 49 49 56 ...
  - features_level_1  (features) int64 59 86 38 46 0 25 19 14 2 9 88 20 38 0 ...
  - features_level_2  (features) float64 2.828 2.828 2.828 2.828 2.828 2.828 ...
  - features_level_3  (features) float64 11.0 17.0 22.0 22.0 29.0 33.0 34.0 ...
  - features_level_4  (features) float64 17.0 23.0 28.0 28.0 35.0 39.0 40.0 ...
  - features_level_5  (features) float64 56.0 83.0 35.0 43.0 0.0 22.0 16.0 ...
  - features_level_6  (features) float64 62.0 89.0 41.0 49.0 3.0 28.0 22.0 ...
  - features_level_7  (features) object nan nan nan nan nan nan nan nan nan ...
  - features_level_8  (features) object nan nan nan nan nan nan nan nan nan ...

In [38]:
np.argmax(min_codes, axis=1)

<xarray.DataArray (features: 18, codes: 19)>
array([[0.155031,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ..., 0.15112 ,      nan,      nan],
       ...,
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan]])
Coordinates:
  * codes             (codes) int64 4 7 10 11 21 27 32 39 46 74 77 85 87 94 ...
  * features          (features) MultiIndex
  - features_level_0  (features) int64 14 20 25 25 32 36 37 42 43 49 49 56 ...
  - features_level_1  (features) int64 59 86 38 46 0 25 19 14 2 9 88 20 38 0 ...
  - features_level_2  (features) float64 2.828 2.828 2.828 2.828 2.828 2.828 ...
  - features_level_3  (features) float64 11.0 17.0 22.0 22.0 29.0 33.0 34.0 ...
  - features_level_4  (features) float64

In [37]:
distances.min('codes')

<xarray.DataArray (features: 18)>
array([0.155031, 0.178009, 0.15112 , 0.166003, 0.098847, 0.222323, 0.228627,
       0.214218, 0.179987, 0.177983, 0.21766 , 0.180043, 0.175726, 0.170241,
       0.291551, 0.171933, 0.288613, 0.229283])
Coordinates:
  * features          (features) MultiIndex
  - features_level_0  (features) int64 14 20 25 25 32 36 37 42 43 49 49 56 ...
  - features_level_1  (features) int64 59 86 38 46 0 25 19 14 2 9 88 20 38 0 ...
  - features_level_2  (features) float64 2.828 2.828 2.828 2.828 2.828 2.828 ...
  - features_level_3  (features) float64 11.0 17.0 22.0 22.0 29.0 33.0 34.0 ...
  - features_level_4  (features) float64 17.0 23.0 28.0 28.0 35.0 39.0 40.0 ...
  - features_level_5  (features) float64 56.0 83.0 35.0 43.0 0.0 22.0 16.0 ...
  - features_level_6  (features) float64 62.0 89.0 41.0 49.0 3.0 28.0 22.0 ...
  - features_level_7  (features) object nan nan nan nan nan nan nan nan nan ...
  - features_level_8  (features) object nan nan nan nan nan nan nan 

In [17]:
selected_codes = np.argmax(selected_codes_index, axis=1)

In [18]:
gene_ids = selected_codes_index.indexes['codes'].values[selected_codes.values]

In [34]:
distances.min('features')

<xarray.DataArray (codes: 19)>
array([0.155031, 0.291551, 0.345697, 0.178009, 0.214218, 0.171933, 0.177983,
       0.098847, 0.228627, 0.170241, 0.21766 , 0.166003, 0.288613, 0.175726,
       0.222323, 0.180043, 0.15112 , 0.357655, 0.179987])
Coordinates:
  * codes    (codes) int64 4 7 10 11 21 27 32 39 46 74 77 85 87 94 103 106 ...

In [32]:
max_codes.max('features')

<xarray.DataArray (codes: 19)>
array([     nan, 0.586083,      nan,      nan,      nan, 0.579017,      nan,
       0.594498, 0.572325,      nan, 0.699881, 0.585424,      nan,      nan,
            nan,      nan, 0.591967, 0.590106, 0.566737])
Coordinates:
  * codes    (codes) int64 4 7 10 11 21 27 32 39 46 74 77 85 87 94 103 106 ...

In [49]:
intensity_table.indexes['features'].set_levels(gene_ids, 'gene', inplace=True)

In [19]:
gene_ids

array([109,  77,  77,   7,  77,  85, 110, 110,  46,  85, 108,  77,  39,
       108,  27,   7,  27,  39])

In [135]:
intensity_table.dims

('features', 'c', 'h')

In [152]:
intensity_table.indexes['features'].to_frame()

r   x  x_max  x_min   y  y_max  \
x  y  r        x_min x_max y_min y_max                                          
97 60 2.828427 94.0  100.0 57.0  63.0   2.828427  97  100.0   94.0  60   63.0   
89 83 2.828427 86.0  92.0  80.0  86.0   2.828427  89   92.0   86.0  83   86.0   
87 4  2.828427 84.0  90.0  1.0   7.0    2.828427  87   90.0   84.0   4    7.0   
73 65 2.828427 70.0  76.0  62.0  68.0   2.828427  73   76.0   70.0  65   68.0   
72 0  4.085506 67.0  77.0  0.0   5.0    4.085506  72   77.0   67.0   0    5.0   
68 38 2.828427 65.0  71.0  35.0  41.0   2.828427  68   71.0   65.0  38   41.0   
56 20 2.828427 53.0  59.0  17.0  23.0   2.828427  56   59.0   53.0  20   23.0   
49 88 2.828427 46.0  52.0  85.0  91.0   2.828427  49   52.0   46.0  88   91.0   
   9  2.828427 46.0  52.0  6.0   12.0   2.828427  49   52.0   46.0   9   12.0   
43 2  2.828427 40.0  46.0  0.0   5.0    2.828427  43   46.0   40.0   2    5.0   
42 14 2.828427 39.0  45.0  11.0  17.0   2.828427  42   45.0   39.0  14   17.0   
37 19 2.828427 34.0  40.0  16.0  22.0   2.828427  37   40.0   34.0  19   22.0   
36 25 2.828427 33.0  39.0  22.0  28.0   2.828427  36   39.0   33.0  25   28.0   
32 0  2.828427 29.0  35.0  0.0   3.0    2.828427  32   35.0   29.0   0    3.0   
25 46 2.828427 22.0  28.0  43.0  49.0   2.828427  25   28.0   22.0  46   49.0   
   38 2.828427 22.0  28.0  35.0  41.0   2.828427  25   28.0   22.0  38   41.0   
20 86 2.828427 17.0  23.0  83.0  89.0   2.828427  20   23.0   17.0  86   89.0   
14 59 2.828427 11.0  17.0  56.0  62.0   2.828427  14   17.0   11.0  59   62.0   

                                        y_min  
x  y  r        x_min x_max y_min y_max         
97 60 2.828427 94.0  100.0 57.0  63.0    57.0  
89 83 2.828427 86.0  92.0  80.0  86.0    80.0  
87 4  2.828427 84.0  90.0  1.0   7.0      1.0  
73 65 2.828427 70.0  76.0  62.0  68.0    62.0  
72 0  4.085506 67.0  77.0  0.0   5.0      0.0  
68 38 2.828427 65.0  71.0  35.0  41.0    35.0  
56 20 2.828427 53.0  59.0  17.0  23.0    17.0  
49 88 2.828427 46.0  52.0  85.0  91.0    85.0  
   9  2.828427 46.0  52.0  6.0   12.0     6.0  
43 2  2.828427 40.0  46.0  0.0   5.0      0.0  
42 14 2.828427 39.0  45.0  11.0  17.0    11.0  
37 19 2.828427 34.0  40.0  16.0  22.0    16.0  
36 25 2.828427 33.0  39.0  22.0  28.0    22.0  
32 0  2.828427 29.0  35.0  0.0   3.0      0.0  
25 46 2.828427 22.0  28.0  43.0  49.0    43.0  
   38 2.828427 22.0  28.0  35.0  41.0    35.0  
20 86 2.828427 17.0  23.0  83.0  89.0    83.0  
14 59 2.828427 11.0  17.0  56.0  62.0    56.0

In [155]:
def append_multiindex_level(multiindex, data, name):
    """stupid thing necessary because pandas doesn't do this"""
    frame = multiindex.to_frame()
    frame[name] = data
    frame.set_index(name, append=True, inplace=True)
    return frame.index

In [157]:
augmented_index = append_multiindex_level(intensity_table.indexes['features'], gene_ids, 'gene_name')

In [166]:
augmented_index.get_level_values('x')

Int64Index([97, 89, 87, 73, 72, 68, 56, 49, 49, 43, 42, 37, 36, 32, 25, 25, 20,
            14],
           dtype='int64', name='x')

In [ ]:
# I'm not sure they like me messing around with indices. I could prefill the index, or hack around with a new constructor.

In [169]:
intensity_table.reindex(indexers={'features': augmented_index})

<xarray.IntensityTable (features: 18, c: 4, h: 4)>
array([[[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]],

       [[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]],

       ...,

       [[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]],

       [[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]])
Coordinates:
  * features   (features) MultiIndex
  - x          (features) int64 97 89 87 73 72 68 56 49 49 43 42 37 36 32 25 ...
  - y          (features) int64 60 83 4 65 0 38 20 88 9 2 14 19 25 0 46 38 86 59
  - r          (features) float64 2.828 2.828 2.828 2.828 4.086 2.828 2.828 ...
  - x_min      (features) float64 94.0 86.0 84.0 70.0 67.0 65.0 53.0 46.0 ...
  - x_max      (features) float64 100.0 92.0 90.0 76.0 77.0 71.0 59.0 52.0 ...

In [167]:
intensity_table.set_index(features=[augmented_index.get_level_values(name) for name in augmented_index.names])

TypeError: unhashable type: 'Int64Index'

## Pandas IntensityTable

In [33]:
intensity_table.groupby('c').apply(np.argmax, axis=0)

<xarray.IntensityTable (c: 4, h: 4)>
array([[12,  0, 12,  3],
       [ 0, 13,  1, 13],
       [14,  1,  8, 17],
       [13,  3, 13,  7]])
Coordinates:
  * h        (h) int64 0 1 2 3
  * c        (c) int64 0 1 2 3

In [34]:
intensity_data

NameError: name 'intensity_data' is not defined

## XArray IntensityTable

In [35]:
# we can transform this into various other data arrays. an xarray DataArray is very easy, because it infers things from the pandas DataFrame
xr.DataArray(intensity_data)

NameError: name 'intensity_data' is not defined

In [36]:
# We can also build this more clearly. The below is the same, but we get to name the index.
xarray_intensities = xr.DataArray(
    data=intensity_data.values, 
    coords=(intensity_data.index, intensity_data.columns),
    dims=('Indices', 'spot_id')
)
xarray_intensities

NameError: name 'intensity_data' is not defined

## AnnData IntensityTable

In [63]:
cols = intensity_data.index.to_frame()
cols.index = np.arange(cols.shape[0])

adata_intensities = ad.AnnData(
    X=intensity_data.values,
    obs=cols,
    var=pd.Series(intensity_data.columns),
)

From here, we can play. 

## Test Apply

In [86]:
xr.apply_ufunc(np.argmax, xarray_intensities, input_core_dims=[['Indices']], kwargs={'axis': 1})

<xarray.DataArray (spot_id: 19)>
array([1, 4, 5, 6, 2, 1, 6, 7, 5, 2, 3, 1, 5, 3, 3, 5, 4, 4, 1])
Coordinates:
  * spot_id  (spot_id) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18

My impression here is that they've done a great job of defining a way to apply arbitrary functions across nd data. The result is a bit tricky at first, but the complexity starts to make sense when you realize all the options that open up when you're operating over more dimensions and might want to broadcast (or reduce!) over a few of them. 

In [ ]:
import xarray as xr

class IntensityTable():
    
    def __init__(intensities, feature_metadata, tile_metadata):
        self._data = xr.DataArray(
            data=intensities, 
            coords=(feature_metadata, tile_metadata)
            dims=('tile_metadata', 'feature_metadata')
        )
    
    def decode(func, codebook):
        pass
    
    def show(background_image):
        pass